In [1]:
import numpy as np
import os
import re
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import datetime
from time import time
import sklearn
from sklearn.model_selection import train_test_split,cross_val_score,KFold,StratifiedKFold,ShuffleSplit,StratifiedShuffleSplit, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import PolynomialFeatures,LabelEncoder,Imputer,RobustScaler, StandardScaler, MinMaxScaler,FunctionTransformer
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.feature_selection import RFE
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.linear_model import ElasticNet
from sklearn.externals import joblib
from sklearn.metrics import mean_squared_error,mean_squared_log_error,r2_score
from sklearn_pandas import DataFrameMapper
from pandas.api.types import is_string_dtype, is_numeric_dtype
from scipy import stats
from scipy.stats import skew,randint
from scipy.special import boxcox1p
from scipy.stats import randint as sp_randint
%matplotlib inline

In [2]:
def print_feature_importances(model,X):
    important_features = pd.Series(data=rf_model.feature_importances_,index=X.columns)
    important_features.sort_values(ascending=False,inplace=True)
    print(important_features.head(50))
    
def get_cat_columns_by_type(df):
    out = []
    for colname,col_values in df.items():
        if is_string_dtype(col_values):
            out.append((colname,'string') )
        elif not is_numeric_dtype(col_values):
            out.append((colname,'categorical') )
    return out       

def get_numeric_columns(df):
    out = []
    for colname,col_values in df.items():
        if is_numeric_dtype(col_values):
            out.append(colname)
    return out       
    
def get_missing_values_percentage(df):
    missing_values_counts_list = df.isnull().sum()
    total_values = np.product(df.shape)
    total_missing = missing_values_counts_list.sum()
    # percent of data that is missing
    return (total_missing/total_values) * 100


def convert_to_str_type(df_in,columns,inplace=False):
    if(inplace):
        df = df_in
    else:
        df = df_in.copy()
        
    for col in columns:
        df[col] = df[col].astype(str)
    return df

def extract_and_drop_target_column(df_in, y_name, inplace=False):
    if(inplace):
        df = df_in
    else:
        df = df_in.copy()
    if not is_numeric_dtype(df[y_name]):
        df[y_name] = df[y_name].cat.codes
        y = df[y_name].values
    else:
        y = df[y_name].copy()
    df.drop([y_name], axis=1, inplace=True)
    return (df,y)

def get_cat_and_numerical_cols(df):
    cat_cols = get_cat_columns_by_type(df)
    cat_cols = [cat_cols[i][0] for i in range(len(cat_cols))]
    num_cols = [col for col in df.columns if col not in cat_cols]
    return cat_cols,num_cols

    
def clean_df(df,pipelines):
    cat_cols, num_cols = get_cat_and_numerical_cols(df)
    dfs = []
    if len(num_cols):
        df1 = df[num_cols]
        print(df1.shape)
        pipelines['pipe_df_missing_num'].fit(df1)
        data1 = pipelines['pipe_df_missing_num'].transform(df1)
        df1 = pd.DataFrame(data1,columns=num_cols)
        dfs.append(df1)
    
    if len(cat_cols):
        df2 = df[cat_cols]
        print(df2.shape)
        pipelines['pipe_df_missing_cat'].fit(df2)
        data2 = pipelines['pipe_df_missing_cat'].transform(df2)     
        df2 = pd.DataFrame(data2,columns=cat_cols)
        dfs.append(df2)
        
        
    return pd.concat(dfs,axis=1)

# adapted from https://github.com/fastai/fastai/blob/master/fastai/structured.py
def process_date_column(df_in, colname, include_time=False, inplace=True, 
                        date_format=None):
    if(inplace):
        df = df_in
    else:
        df = df_in.copy()
        
    prefix_without_date = re.sub('[Dd]ate$', '', colname)
    if(df[colname].dtype != 'datetime64[ns]'):
        if date_format is not None:
            df[colname] = pd.to_datetime(df[colname],format=date_format)
        else:
            df[colname] = pd.to_datetime(df[colname],infer_datetime_format=True)
    columns = ['Year', 'Month', 'Week','Day',
               'Dayofweek', 'Dayofyear',
               'Is_month_end','Is_month_start',
               'Is_quarter_end','Is_quarter_start',
               'Is_year_end','Is_year_start']
    if include_time:
        columns = columns + ['Hour', 'Minute', 'Second']
    for c in columns:
        df[prefix_without_date + '_' + c] = getattr(df[colname].dt,c.lower())
    df[prefix_without_date] = df[colname].astype(np.int64) // (10 ** 9)
    df.drop(colname,axis=1,inplace=True)
    return df

def handle_encoding(df,one_hot=False,ignore_columns=None):
    lbl = LabelEncoder()
    cat_cols,_ = get_cat_and_numerical_cols(df)
    print('len of cat cols = {}'.format(len(cat_cols)))
    for colname in cat_cols:
        if ignore_columns is not None:
            if colname in ignore_columns:
                continue
        #print(colname)
        lbl.fit(list(df[colname].values)) 
        df[colname] = lbl.transform(list(df[colname].values))
        
    if one_hot:
        return pd.get_dummies(df,columns=cat_cols,dummy_na=True)
    else:
        return df
    

def get_iqr_min_max(df,cols):
    out = {}
    for colname, col_values in df.items():
        if colname not in cols:
            continue
        quartile75, quartile25 = np.percentile(col_values, [75 ,25])
        ## Inter Quartile Range ##
        IQR = quartile75 - quartile25
        min_value = quartile25 - (IQR*1.5)
        max_value = quartile75 + (IQR*1.5)
        out[colname] = (min_value,max_value)
    return out

def remove_skew(df,threshold=0.75,lambda_in=0.15):
    cat_cols, num_cols = get_cat_and_numerical_cols(df)
    skewed_cols = df[num_cols].apply(lambda x: skew(x)).sort_values(ascending=False)
    skewness = pd.DataFrame({'Skew' :skewed_cols})
    skewness_log = skewness[abs(skewness) > threshold]
    skewness_other = skewness[abs(skewness) <= threshold]
    skewed_features_log = skewness_log.index
    skewed_features_other = skewness_other.index
    lambda_ = 0.0
    for feature in skewed_features_log:
        df[feature] = boxcox1p(df[feature],lambda_)
        lambda_ = lambda_in
    for feature in skewed_features_other:
        df[feature] = boxcox1p(df[feature],lambda_)
    return df

def bin_numerical_columns(df_in,cols,inplace=False):
    if(inplace):
        df = df_in
    else:
        df = df_in.copy()
        
    for col in cols.keys():
        bins = cols[col]
        buckets_ = np.linspace(bins[0],bins[1],bins[2])
        df[col] = pd.cut(df[col],buckets_,include_lowest=True)
    return df

# Utility function to report best scores
def report_best_scores(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [3]:

def preprocess_df2(df,id_col= None,df_test=None,test_id=None,
                   new_features_func=None,
                   date_col=None,
                   convert_to_cat_cols=None,
                   bin_columns_dict=None,
                   remove_skewness=False,
                   skew_threshold=0.75,
                   boxcox_lambda=0.15
                  ):
    
            
    if id_col is not None:
        combined.drop(id_col, axis=1,inplace=True)
        if df_test is not None and test_id is not None:
            test_id = df_test[id_col].copy()
        else: test_id = None
           
   
    if new_features_func is not None:
        df = new_features_func(df)
    
    if date_col is not None:
        process_date_column(df,colname=date_col)
        
    if convert_to_cat_cols is not None:
        df = convert_to_str_type(df,convert_to_cat_cols,inplace=True)
        
    if bin_columns_dict is not None:
        df = bin_numerical_columns(df,bin_columns_dict,inplace=True)
    
    return df,test_id

def create_cleaning_pipelines(log_y=False,one_hot=False):
    def log_of_y(y):
        if log_y:
            return np.log1p(y)
        else: 
            return y
        
    pipeline_y = make_pipeline(FunctionTransformer(log_of_y))
    pipeline_df_missing_num = make_pipeline(Imputer(strategy='median',axis=0))
    pipeline_df_missing_cat = make_pipeline(Imputer(strategy='most_frequent',axis=0)
                                            
                                            )
    
    return {'pipe_y':pipeline_y,
            'pipe_df_missing_num':pipeline_df_missing_num,
            'pipe_df_missing_cat':pipeline_df_missing_cat
           }



In [5]:
def add_new_features1(df):
    return df
def add_new_features2(df):
    return df


In [6]:
PATH = "data/bulldozers/"
df_raw = pd.read_csv(f'{PATH}train.csv', low_memory=False,
                     parse_dates=["saledate"])
df_test = pd.read_csv(f'{PATH}test.csv', low_memory=False,parse_dates=['saledate'])

df_raw.sort_values('saledate',inplace=True)
df_test.sort_values('saledate',inplace=True)

In [7]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 401125 entries, 205615 to 400217
Data columns (total 53 columns):
SalesID                     401125 non-null int64
SalePrice                   401125 non-null int64
MachineID                   401125 non-null int64
ModelID                     401125 non-null int64
datasource                  401125 non-null int64
auctioneerID                380989 non-null float64
YearMade                    401125 non-null int64
MachineHoursCurrentMeter    142765 non-null float64
UsageBand                   69639 non-null object
saledate                    401125 non-null datetime64[ns]
fiModelDesc                 401125 non-null object
fiBaseModel                 401125 non-null object
fiSecondaryDesc             263934 non-null object
fiModelSeries               56908 non-null object
fiModelDescriptor           71919 non-null object
ProductSize                 190350 non-null object
fiProductClassDesc          401125 non-null object
state           

In [8]:
df_raw['UsageBand'].value_counts()

Medium    33985
Low       23620
High      12034
Name: UsageBand, dtype: int64

In [9]:
df = df_raw.copy()
df,y = extract_and_drop_target_column(df,'SalePrice',inplace=True)

n_train = df.shape[0]
n_test = df_test.shape[0]
    
pipelines = create_cleaning_pipelines(log_y=True)

combined = pd.concat((df, df_test)).reset_index(drop=True)


c:\program files\python37\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
c:\program files\python37\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [10]:
combined,test_id = preprocess_df2(combined,id_col='SalesID',
                                    df_test=df_test,test_id='SalesID',
                                    convert_to_cat_cols=['sale_Year', 
                                                         'sale_Month',
                                                         'sale_Week',
                                                         'sale_Day',
                                                         'sale_Dayofweek',
                                                         'sale_Dayofyear',
                                                         'UsageBand'],
                                    date_col='saledate'
                                    
                                   )

combined = handle_encoding(combined,one_hot=False,ignore_columns=[
                                                         'sale_Is_month_end',
                                                         'sale_Is_month_start',
                                                         'sale_Is_quarter_end',
                                                         'sale_Is_quarter_start',
                                                         'sale_Is_year_end',
                                                         'sale_Is_year_start'
                                                          ])
print('combined shape = {}'.format(combined.shape) )
combined = clean_df(combined,pipelines)

combined = remove_skew(combined,threshold=0.75,lambda_in=0.15)
print(get_missing_values_percentage(combined))

y = clean_df(pd.DataFrame(y),pipelines)
y = pipelines['pipe_y'].fit_transform(y)
y = pd.DataFrame(y)
print(get_missing_values_percentage(y))
print(y.head())
y = y.values.ravel()
print(y.shape)
df = combined[:n_train]
df_test = combined[n_train:]

len of cat cols = 50
combined shape = (413582, 63)
(413582, 63)
0.0
(401125, 1)
0.0
           0
0   9.159152
1  10.085851
2  10.463132
3   9.852247
4   9.546884
(401125,)


c:\program files\python37\lib\site-packages\sklearn\preprocessing\_function_transformer.py:98: FutureWarning: The default validate=True will be replaced by validate=False in 0.22.
  "validate=False in 0.22.", FutureWarning)
c:\program files\python37\lib\site-packages\sklearn\preprocessing\_function_transformer.py:98: FutureWarning: The default validate=True will be replaced by validate=False in 0.22.
  "validate=False in 0.22.", FutureWarning)


In [11]:
df_raw.shape,df.shape,df_test.shape

((401125, 53), (401125, 63), (12457, 63))

In [12]:
df['sale_Is_month_end'].head()

0    0.000000
1    0.571575
2    0.571575
3    0.571575
4    0.571575
Name: sale_Is_month_end, dtype: float64

In [13]:
df['UsageBand'].value_counts()

1.000916    331486
0.834066     33985
0.571575     23620
0.000000     12034
Name: UsageBand, dtype: int64

In [14]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12457 entries, 401125 to 413581
Data columns (total 63 columns):
MachineID                   12457 non-null float64
ModelID                     12457 non-null float64
datasource                  12457 non-null float64
auctioneerID                12457 non-null float64
YearMade                    12457 non-null float64
MachineHoursCurrentMeter    12457 non-null float64
UsageBand                   12457 non-null float64
fiModelDesc                 12457 non-null float64
fiBaseModel                 12457 non-null float64
fiSecondaryDesc             12457 non-null float64
fiModelSeries               12457 non-null float64
fiModelDescriptor           12457 non-null float64
ProductSize                 12457 non-null float64
fiProductClassDesc          12457 non-null float64
state                       12457 non-null float64
ProductGroup                12457 non-null float64
ProductGroupDesc            12457 non-null float64
Drive_System      

In [15]:
combined.head()

,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,fiModelDesc,fiBaseModel,fiSecondaryDesc,...,sale_Day,sale_Dayofweek,sale_Dayofyear,sale_Is_month_end,sale_Is_month_start,sale_Is_quarter_end,sale_Is_quarter_start,sale_Is_year_end,sale_Is_year_start,sale
0,5.257358,3.798039,2.476808,1.742457,3.354912,0.0,1.000916,3.615342,3.318084,2.580998,...,1.413141,0.571575,2.287819,0.000000,0.0,0.0,0.0,0.0,0.0,7.210275
1,5.268730,3.746500,2.476808,2.376714,3.356789,0.0,1.000916,3.050685,2.682502,2.580998,...,1.854993,0.571575,2.669830,0.571575,0.0,0.0,0.0,0.0,0.0,7.210929
2,5.291483,3.518361,2.476808,2.376714,3.355069,0.0,1.000916,2.310205,1.742457,2.192041,...,1.854993,0.571575,2.669830,0.571575,0.0,0.0,0.0,0.0,0.0,7.210929
3,5.329659,3.791279,2.476808,2.376714,3.355539,0.0,1.000916,3.194731,2.803758,2.580998,...,1.854993,0.571575,2.669830,0.571575,0.0,0.0,0.0,0.0,0.0,7.210929
4,5.274867,3.853804,2.476808,2.376714,3.355852,0.0,1.000916,3.331938,2.956367,2.580998,...,1.854993,0.571575,2.669830,0.571575,0.0,0.0,0.0,0.0,0.0,7.210929


In [16]:
df_test.head()

,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,fiModelDesc,fiBaseModel,fiSecondaryDesc,...,sale_Day,sale_Dayofweek,sale_Dayofyear,sale_Is_month_end,sale_Is_month_start,sale_Is_quarter_end,sale_Is_quarter_start,sale_Is_year_end,sale_Is_year_start,sale
401125,5.471426,3.787044,2.567451,0.571575,3.358654,3.598102,0.834066,3.088665,2.708977,2.580998,...,0.0,0.571575,1.870771,0.0,0.571575,0.0,0.0,0.0,0.0,7.472117
401126,5.388952,3.501581,2.567451,0.571575,3.358189,3.607050,0.571575,3.171819,2.779307,2.489621,...,0.0,0.571575,1.870771,0.0,0.571575,0.0,0.0,0.0,0.0,7.472117
401127,5.469791,4.027418,2.567451,0.571575,3.359582,3.377866,0.571575,3.340780,2.981315,2.580998,...,0.0,0.571575,1.870771,0.0,0.571575,0.0,0.0,0.0,0.0,7.472117
401128,5.470566,3.554883,2.567451,0.571575,3.359119,3.821435,0.834066,3.314160,2.932371,2.192041,...,0.0,0.571575,1.870771,0.0,0.571575,0.0,0.0,0.0,0.0,7.472117
401129,5.472091,3.999370,2.567451,0.571575,3.359891,3.103710,0.571575,3.556413,3.246907,2.365904,...,0.0,0.571575,1.870771,0.0,0.571575,0.0,0.0,0.0,0.0,7.472117


In [17]:
combined.head()

,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,fiModelDesc,fiBaseModel,fiSecondaryDesc,...,sale_Day,sale_Dayofweek,sale_Dayofyear,sale_Is_month_end,sale_Is_month_start,sale_Is_quarter_end,sale_Is_quarter_start,sale_Is_year_end,sale_Is_year_start,sale
0,5.257358,3.798039,2.476808,1.742457,3.354912,0.0,1.000916,3.615342,3.318084,2.580998,...,1.413141,0.571575,2.287819,0.000000,0.0,0.0,0.0,0.0,0.0,7.210275
1,5.268730,3.746500,2.476808,2.376714,3.356789,0.0,1.000916,3.050685,2.682502,2.580998,...,1.854993,0.571575,2.669830,0.571575,0.0,0.0,0.0,0.0,0.0,7.210929
2,5.291483,3.518361,2.476808,2.376714,3.355069,0.0,1.000916,2.310205,1.742457,2.192041,...,1.854993,0.571575,2.669830,0.571575,0.0,0.0,0.0,0.0,0.0,7.210929
3,5.329659,3.791279,2.476808,2.376714,3.355539,0.0,1.000916,3.194731,2.803758,2.580998,...,1.854993,0.571575,2.669830,0.571575,0.0,0.0,0.0,0.0,0.0,7.210929
4,5.274867,3.853804,2.476808,2.376714,3.355852,0.0,1.000916,3.331938,2.956367,2.580998,...,1.854993,0.571575,2.669830,0.571575,0.0,0.0,0.0,0.0,0.0,7.210929


In [18]:
processing_pipeline1 = make_pipeline(RobustScaler(),
                                     StandardScaler(),
                                     RandomForestRegressor())

processing_pipeline2 = make_pipeline(RobustScaler(),
                                     StandardScaler(),
                                     GradientBoostingRegressor())

In [19]:
df.shape,df_test.shape,y.shape

((401125, 63), (12457, 63), (401125,))

In [20]:
def simple_split(df,y,n):
    X_train =  df[:n].copy()
    X_test = df[n:].copy()
    y_train = y[:n].copy()
    y_test  = y[n:].copy()
    return X_train,X_test,y_train,y_test


In [21]:
test_size = 12000

In [22]:
X_train,X_test,y_train,y_test = simple_split(df,y,(df.shape[0] - test_size))
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)
X_train,X_valid,y_train,y_valid = simple_split(X_train,y_train,
                                               X_train.shape[0] - test_size)
print(X_train.shape,X_valid.shape,y_train.shape,y_valid.shape)

(389125, 63) (12000, 63) (389125,) (12000,)
(377125, 63) (12000, 63) (377125,) (12000,)


In [23]:
def print_mse(m,X_train, X_valid, y_train, y_valid):
    res = [mean_squared_error(y_train,m.predict(X_train)),
                mean_squared_error(y_valid,m.predict(X_valid)),
                m.score(X_train, y_train), m.score(X_valid, y_valid)]
    print('MSE Training set = {}, MSE Validation set = {}, score Training Set = {}, score on Validation Set = {}'.format(res[0],res[1],res[2], res[3]))
    if hasattr(m, 'oob_score_'):
          print('OOB Score = {}'.format(m.oob_score_)) 

In [24]:
pipe_model1 = processing_pipeline1.fit(X_train,y_train)
print_mse(pipe_model1, X_train,X_valid,y_train,y_valid)

c:\program files\python37\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


MSE Training set = 0.008286481248334612, MSE Validation set = 0.06775199309248264, score Training Set = 0.9826909889024695, score on Validation Set = 0.865126891374754


In [25]:
pipe_model2 = processing_pipeline2.fit(X_train,y_train)
print_mse(pipe_model2, X_train,X_valid,y_train,y_valid)

MSE Training set = 0.0979350956025239, MSE Validation set = 0.10163424007752185, score Training Set = 0.7954307014255927, score on Validation Set = 0.7976778943859459


In [26]:
params = {'randomforestregressor__n_estimators':[10,20,40,60],
              "randomforestregressor__max_features": randint(10,64),
              "randomforestregressor__min_samples_split": randint(2, 11),
              "randomforestregressor__min_samples_leaf": randint(1, 11)
         }

start = time()
randomSearch_p1 = RandomizedSearchCV(processing_pipeline1,
                                     param_distributions=params,
                                     n_iter=10,n_jobs=6,
                                     scoring='neg_mean_squared_error'
                                     ).fit(X_train,y_train)

print('training took {} mins'.format((time() - start)/60.))

c:\program files\python37\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


training took 9.865293117364248 mins


In [27]:
report_best_scores(randomSearch_p1.cv_results_)

Model with rank: 1
Mean validation score: -0.131 (std: 0.059)
Parameters: {'randomforestregressor__max_features': 15, 'randomforestregressor__min_samples_leaf': 1, 'randomforestregressor__min_samples_split': 7, 'randomforestregressor__n_estimators': 60}

Model with rank: 2
Mean validation score: -0.133 (std: 0.058)
Parameters: {'randomforestregressor__max_features': 16, 'randomforestregressor__min_samples_leaf': 6, 'randomforestregressor__min_samples_split': 5, 'randomforestregressor__n_estimators': 60}

Model with rank: 3
Mean validation score: -0.133 (std: 0.057)
Parameters: {'randomforestregressor__max_features': 12, 'randomforestregressor__min_samples_leaf': 4, 'randomforestregressor__min_samples_split': 7, 'randomforestregressor__n_estimators': 60}



In [28]:
processing_pipeline_rf = make_pipeline(RobustScaler(),
                                     StandardScaler(),
                                     RandomForestRegressor(n_estimators=60,
                                                          max_features=40,
                                                          min_samples_leaf=2,
                                                          min_samples_split=4))

In [29]:
pipe_model_rf = processing_pipeline_rf.fit(X_train,y_train)
print_mse(pipe_model_rf, X_train,X_valid,y_train,y_valid)

MSE Training set = 0.009778704429481018, MSE Validation set = 0.05713352877195597, score Training Set = 0.9795739954732445, score on Validation Set = 0.8862649454210872


In [30]:
joblib.dump(pipe_model_rf,'rf_model_rank1_bulldozers.pkl')

['rf_model_rank1_bulldozers.pkl']